In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.0.2
3.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x000002269CBC13A0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x000002269CBC13A0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x000002269CBC13A0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x000002269CBC13A0>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x000002269CBC13A0>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x000002269CBC13A0>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [8]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:15<00:00, 12.52it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,69,221.023376,10.0,0.612853,225.411013,1.5,0.278286,90.425356,1.0,0.930530,...,10,7,3,6,5,3,0,4,0,dikesnet
1,59,120.857354,1.5,0.742109,127.198807,1.5,0.644897,158.422428,1.0,0.176009,...,10,7,3,6,5,3,0,5,0,dikesnet
2,52,329.111934,10.0,0.295050,244.208683,10.0,0.239674,345.218592,1.5,0.291367,...,10,7,3,6,5,3,0,6,0,dikesnet
3,21,296.878160,1.5,0.579170,266.143568,1.5,0.871824,293.761080,10.0,0.040766,...,10,7,3,6,5,3,0,7,0,dikesnet
4,82,93.920649,1.5,0.228620,313.311091,1.0,0.634719,332.892563,1.0,0.987247,...,10,7,3,6,5,3,0,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,44,108.830072,10.0,0.814868,233.576530,1.0,0.391640,187.459663,1.0,0.386501,...,6,1,6,2,1,5,2,49,3,dikesnet
196,11,48.232595,10.0,0.309814,241.076030,1.5,0.481866,240.058592,1.5,0.378285,...,6,1,6,2,1,5,2,50,3,dikesnet
197,115,52.934279,1.0,0.060758,122.440710,1.0,0.563679,226.652336,1.5,0.012291,...,6,1,6,2,1,5,2,51,3,dikesnet
198,72,161.156649,1.0,0.785318,32.693343,1.5,0.172013,46.484232,1.0,0.447739,...,6,1,6,2,1,5,2,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.493124e+08,0.0,1.984829e+08,0.010178,1.068026e+08,0.000000,5.530705e+07,0.0,1.290658e+08,0.000000,8.564000e+08,0.000000
1,2.493124e+08,0.0,1.914127e+08,0.001730,1.068026e+08,0.000000,5.530705e+07,0.0,1.290658e+08,0.000000,8.564000e+08,0.000000
2,2.493124e+08,0.0,1.963108e+08,0.007618,1.068026e+08,0.000000,5.530705e+07,0.0,1.290658e+08,0.000000,8.564000e+08,0.000000
3,2.493124e+08,0.0,1.895730e+08,0.000000,1.080112e+08,0.001600,5.530705e+07,0.0,1.290658e+08,0.000000,8.564000e+08,0.000000
4,2.493124e+08,0.0,1.919475e+08,0.001714,1.068026e+08,0.000000,5.530705e+07,0.0,1.290658e+08,0.000000,8.564000e+08,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.698586e+08,0.0,1.490725e+08,0.000631,1.361440e+08,0.000000,3.624297e+07,0.0,9.492220e+07,0.000000,1.321100e+09,184.346880
196,2.698586e+08,0.0,1.473184e+08,0.000273,1.361440e+08,0.000000,3.624297e+07,0.0,9.499398e+07,0.000010,1.321100e+09,82.795281
197,2.698586e+08,0.0,1.448688e+08,0.000000,1.389661e+08,0.000946,3.624297e+07,0.0,1.598319e+08,0.009682,1.321100e+09,3954.687313
198,2.698586e+08,0.0,1.645779e+08,0.002473,1.361440e+08,0.000000,3.624297e+07,0.0,9.622138e+07,0.000133,1.321100e+09,789.441455


In [11]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [12]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [00:25<00:00, 11.67it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [13]:
experiments, outcomes = results

In [14]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,4.620660e+06,0.005443,4.152414e+07,0.090124,2.824868e+07,0.016040,5.315896e+07,0.056448,253800000.0,0.0
1,5.397251e+07,0.000000,9.875764e+07,0.096251,1.476749e+09,2.594713,1.061808e+07,0.004789,1.715375e+06,0.001558,253800000.0,0.0
2,5.397251e+07,0.000000,4.682906e+07,0.050271,2.386701e+08,0.473904,5.485795e+07,0.027997,0.000000e+00,0.000000,253800000.0,0.0
3,5.397251e+07,0.000000,5.559100e+07,0.050168,2.106536e+08,0.353636,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,9.558758e+06,0.011308,1.141946e+09,2.315536,6.373503e+06,0.003377,0.000000e+00,0.000000,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.625123e+07,0.009414,4.625496e+07,0.037693,2.879840e+07,0.000000,1.218871e+07,0.005005,0.000000e+00,0.000000,369700000.0,0.0
296,1.341257e+09,0.837254,1.460801e+06,0.001369,2.879840e+07,0.000000,1.188652e+07,0.005491,0.000000e+00,0.000000,369700000.0,0.0
297,2.975870e+09,1.660996,7.115702e+07,0.057468,2.879840e+07,0.000000,2.301570e+07,0.009150,0.000000e+00,0.000000,369700000.0,0.0
298,0.000000e+00,0.000000,7.561023e+08,0.597678,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
